In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

import plotly 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
mpl.rc('font', family='Apple SD Gothic Neo')
mpl.rcParams['axes.unicode_minus'] = False

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit


import optuna
from optuna.samplers import TPESampler

from joblib import Parallel, delayed,dump
import pickle

In [5]:
## SMAPE
def smape(y_true, y_pred):
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))
    )

In [6]:
train_df = pd.read_csv('train2.csv')
valid_df = pd.read_csv('valid2.csv')
test_df = pd.read_csv('test2.csv')

In [8]:
train_df['august'] = (train_df['월'] == 8).astype(int)
valid_df['august'] = (valid_df['월'] == 8).astype(int)
test_df['august'] = (test_df['월'] == 8).astype(int)

In [10]:
drop_col_for_each = ['datetime','월','일','연면적(m2)','냉방면적(m2)',
                '태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)','건물유형','면적비율']

In [11]:
train_all = train_df.drop(drop_col_for_each,axis=1)
valid_all = valid_df.drop(drop_col_for_each,axis=1)
test_all = test_df.drop(drop_col_for_each,axis=1)

In [12]:
for df in [train_all,valid_df, test_all]:
    df["weekend"] = df["weekend"].astype("category")
    df['요일'] = df["요일"].astype("category")
    df['시간'] = df["시간"].astype("category")
    df['august'] = df['august'].astype('category')

In [13]:
train_all

,건물번호,기온(°C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),시간,요일,weekend,hour_sin,hour_cos,dow_sin,dow_cos,hour_sin_dow_sin,hour_sin_dow_cos,hour_cos_dow_sin,hour_cos_dow_cos,불쾌지수,august
0,1,18.3,0.0,2.6,82.0,5794.80,0,5,1,0.000000,1.000000,-0.974928,-0.222521,-0.000000,-0.000000,-0.974928,-0.222521,17.92380,0
1,1,18.3,0.0,2.7,82.0,5591.85,1,5,1,0.258819,0.965926,-0.974928,-0.222521,-0.252330,-0.057593,-0.941708,-0.214939,17.92380,0
2,1,18.1,0.0,2.6,80.0,5338.17,2,5,1,0.500000,0.866025,-0.974928,-0.222521,-0.487464,-0.111260,-0.844312,-0.192709,17.70400,0
3,1,18.0,0.0,2.6,81.0,4554.42,3,5,1,0.707107,0.707107,-0.974928,-0.222521,-0.689378,-0.157346,-0.689378,-0.157346,17.63425,0
4,1,17.8,0.0,1.3,81.0,3602.25,4,5,1,0.866025,0.500000,-0.974928,-0.222521,-0.844312,-0.192709,-0.487464,-0.111260,17.45515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187195,100,29.1,0.0,2.8,72.0,2667.72,19,5,1,-0.965926,0.258819,-0.974928,-0.222521,0.941708,0.214939,-0.252330,-0.057593,26.85160,1
187196,100,28.6,0.0,3.2,75.0,1994.88,20,5,1,-0.866025,0.500000,-0.974928,-0.222521,0.844312,0.192709,-0.487464,-0.111260,26.66125,1
187197,100,28.4,0.0,2.5,75.0,2177.76,21,5,1,-0.707107,0.707107,-0.974928,-0.222521,0.689378,0.157346,-0.689378,-0.157346,26.48875,1
187198,100,28.2,0.0,2.4,76.0,2911.32,22,5,1,-0.500000,0.866025,-0.974928,-0.222521,0.487464,0.111260,-0.844312,-0.192709,26.39160,1


In [15]:
valid_all

,건물번호,기온(°C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),시간,요일,weekend,hour_sin,hour_cos,dow_sin,dow_cos,hour_sin_dow_sin,hour_sin_dow_cos,hour_cos_dow_sin,hour_cos_dow_cos,불쾌지수,august
0,1,27.3,0.0,1.5,83.0,4036.38,0,6,1,0.000000,1.000000,-0.781831,0.623490,-0.000000,0.000000,-0.781831,0.623490,26.10320,1
1,1,27.3,0.0,1.2,84.0,3923.43,1,6,1,0.258819,0.965926,-0.781831,0.623490,-0.202353,0.161371,-0.755191,0.602245,26.17360,1
2,1,27.2,0.0,2.3,85.0,3917.10,2,6,1,0.500000,0.866025,-0.781831,0.623490,-0.390916,0.311745,-0.677086,0.539958,26.15225,1
3,1,27.1,0.0,2.1,84.0,3875.88,3,6,1,0.707107,0.707107,-0.781831,0.623490,-0.552838,0.440874,-0.552838,0.440874,25.99120,1
4,1,27.1,0.0,1.6,83.0,4023.48,4,6,1,0.866025,0.500000,-0.781831,0.623490,-0.677086,0.539958,-0.390916,0.311745,25.92190,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,29.1,0.0,4.4,76.0,3276.00,19,5,1,-0.965926,0.258819,-0.974928,-0.222521,0.941708,0.214939,-0.252330,-0.057593,27.17280,1
16796,100,28.6,0.0,3.7,74.0,3197.52,20,5,1,-0.866025,0.500000,-0.974928,-0.222521,0.844312,0.192709,-0.487464,-0.111260,26.58370,1
16797,100,28.3,0.0,2.9,74.0,3006.60,21,5,1,-0.707107,0.707107,-0.974928,-0.222521,0.689378,0.157346,-0.689378,-0.157346,26.32660,1
16798,100,28.0,0.0,1.7,76.0,2649.72,22,5,1,-0.500000,0.866025,-0.974928,-0.222521,0.487464,0.111260,-0.844312,-0.192709,26.21800,1


---

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import xgboost as xgb

In [26]:
import numpy as np
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
import json, os

SAVE_PATH = "xgb_val_100.json"

# ===== SMAPE =====
def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8)
    )

# ===== 건물별 튜닝 (valid_all 고정 검증) =====
def tune_building(b_id):
    print(f"\n=== [건물 {b_id}] 튜닝 시작 ===")

    # --- 학습/검증 데이터 분리 ---
    bld_train = train_all[train_all["건물번호"] == b_id].copy()
    bld_valid = valid_all[valid_all["건물번호"] == b_id].copy()

    # 유효성 체크
    if bld_train.empty or bld_valid.empty:
        print(f"[건물 {b_id}] train/valid 중 하나가 비어 있습니다. 스킵.")
        result = {"best_params": {}, "best_value": float("inf")}
        _save_result(SAVE_PATH, b_id, result)
        return int(b_id), {}, float("inf")

    X_train = bld_train.drop(columns=["건물번호", "전력소비량(kWh)"])
    y_train = bld_train["전력소비량(kWh)"].astype(float)

    X_val = bld_valid.drop(columns=["건물번호", "전력소비량(kWh)"])
    y_val = bld_valid["전력소비량(kWh)"].astype(float)

    def objective(trial):
        params = {
            "objective": "reg:absoluteerror",
            "tree_method": "hist",
            "enable_categorical": True,
            # "max_depth": trial.suggest_int("max_depth", 2, 60),
            "max_leaves": trial.suggest_int("max_leaves", 5, 1000),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            # "min_child_weight": trial.suggest_float("min_child_weight", 1e-4, 50, log=True),
            "lambda": trial.suggest_float("lambda", 1e-2, 100, log=True),
            "alpha": trial.suggest_float("alpha", 1e-2, 10, log=True),
            "max_bin": trial.suggest_int("max_bin", 64, 2000),
            "n_estimators": trial.suggest_int("n_estimators", 30, 5000),
            "random_state": 7 + int(b_id),
        }

        # 학습: train_all 전체로 fit
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train)

        # 검증: valid_all에 대해 SMAPE
        preds = model.predict(X_val)
        return smape(y_val, preds)

    sampler = TPESampler(seed= 6+int(b_id), n_startup_trials=10, multivariate=True)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=100, n_jobs=1)

    best_params_clean = {
        k: (float(v) if isinstance(v, (np.floating,)) else
            int(v) if isinstance(v, (np.integer,)) else v)
        for k, v in study.best_params.items()
    }
    result = {
        "best_params": best_params_clean,
        "best_value": float(study.best_value)
    }

    _save_result(SAVE_PATH, b_id, result)
    print(f"=== [건물 {b_id}] 완료 → Best SMAPE (valid): {study.best_value:.4f} ===")
    return int(b_id), best_params_clean, float(study.best_value)

# ===== JSON 저장 유틸 =====
def _save_result(path, b_id, result):
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                prev_data = json.load(f)
        except json.JSONDecodeError:
            prev_data = {}
    else:
        prev_data = {}
    prev_data[str(b_id)] = result
    with open(path, "w", encoding="utf-8") as f:
        json.dump(prev_data, f, indent=2, ensure_ascii=False)


In [ ]:
# ===== 병렬 실행 =====
p_core_count = 8
building_ids = train_all["건물번호"].unique()

results = Parallel(n_jobs=p_core_count, verbose=10)(
    delayed(tune_building)(b_id) for b_id in building_ids
)

print("\n=== 전체 결과 ===")
for b_id, best_params, best_value in results:
    print(f"건물 {b_id} → {best_params}, SMAPE: {best_value:.2f}")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.



=== [건물 30] 튜닝 시작 ===

=== [건물 32] 튜닝 시작 ===

=== [건물 27] 튜닝 시작 ===

=== [건물 29] 튜닝 시작 ===

=== [건물 26] 튜닝 시작 ===

=== [건물 28] 튜닝 시작 ===

=== [건물 25] 튜닝 시작 ===

=== [건물 31] 튜닝 시작 ===


/opt/anaconda3/lib/python3.13/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-08-18 03:09:10,767] A new study created in memory with name: no-name-8e08daf3-62de-4f5c-b1c1-ae849f0292ce
/opt/anaconda3/lib/python3.13/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-08-18 03:09:10,767] A new study created in memory with name: no-name-8bd11e82-9669-4ddf-ad5c-aa2e04131e42
/opt/anaconda3/lib/python3.13/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-08-18 03:09:10,769] A new study created in memory with name: no-name-d40f4762-6fac-489d-a521-edd2b00b4073
/opt/anaconda3/lib/python3.13/site-packages/op

=== [건물 25] 완료 → Best SMAPE (valid): 11.1420 ===

=== [건물 33] 튜닝 시작 ===


[I 2025-08-18 03:13:33,313] Trial 51 finished with value: 3.667116921150389 and parameters: {'max_leaves': 910, 'learning_rate': 0.013237261716586079, 'subsample': 0.808558459723905, 'colsample_bytree': 0.8173206988072186, 'lambda': 0.6379455966655505, 'alpha': 0.057058801995263655, 'max_bin': 927, 'n_estimators': 4945}. Best is trial 7 with value: 3.648547765249557.
[I 2025-08-18 03:13:34,019] Trial 51 finished with value: 8.906243001926002 and parameters: {'max_leaves': 639, 'learning_rate': 0.001251658306761387, 'subsample': 0.8108959920256001, 'colsample_bytree': 0.9686074705096632, 'lambda': 0.41037781019319963, 'alpha': 0.13914861822428706, 'max_bin': 1372, 'n_estimators': 2902}. Best is trial 44 with value: 8.319659975617975.
[I 2025-08-18 03:13:35,142] Trial 84 finished with value: 11.102731417307151 and parameters: {'max_leaves': 185, 'learning_rate': 0.0028548746233992366, 'subsample': 0.6250361308821313, 'colsample_bytree': 0.9702556468838892, 'lambda': 0.013540984652500863,

...

[I 2025-08-18 04:16:20,113] Trial 97 finished with value: 14.998163114488039 and parameters: {'max_leaves': 412, 'learning_rate': 0.0051128621483020826, 'subsample': 0.7869759040483226, 'colsample_bytree': 0.8748130773254342, 'lambda': 0.22702636752448407, 'alpha': 2.986208991218738, 'max_bin': 692, 'n_estimators': 4240}. Best is trial 86 with value: 14.360431804869656.
[I 2025-08-18 04:16:22,589] Trial 98 finished with value: 14.974587279406496 and parameters: {'max_leaves': 284, 'learning_rate': 0.00921136788440772, 'subsample': 0.7386854687266269, 'colsample_bytree': 0.9454356943946562, 'lambda': 1.2557249209858838, 'alpha': 9.460941081400517, 'max_bin': 790, 'n_estimators': 3674}. Best is trial 86 with value: 14.360431804869656.
[I 2025-08-18 04:16:22,597] Trial 98 finished with value: 2.909531929251337 and parameters: {'max_leaves': 62, 'learning_rate': 0.008792269194608205, 'subsample': 0.5582660838535785, 'colsample_bytree': 0.9590043349174086, 'lambda': 0.05020476956801927, 'al

=== [건물 99] 완료 → Best SMAPE (valid): 2.4178 ===


[I 2025-08-18 04:16:26,563] Trial 99 finished with value: 14.595646671741497 and parameters: {'max_leaves': 402, 'learning_rate': 0.0010713727160065795, 'subsample': 0.6170038091195617, 'colsample_bytree': 0.9517835123012919, 'lambda': 75.29822763031152, 'alpha': 0.7254945839358464, 'max_bin': 1337, 'n_estimators': 4915}. Best is trial 86 with value: 14.360431804869656.
[Parallel(n_jobs=8)]: Done  76 out of  76 | elapsed: 67.3min finished


=== [건물 100] 완료 → Best SMAPE (valid): 14.3604 ===

=== 전체 결과 ===
건물 25 → {'max_leaves': 948, 'learning_rate': 0.042303790105255985, 'subsample': 0.8144844764183835, 'colsample_bytree': 0.8192994213758938, 'lambda': 0.2425526040052131, 'alpha': 9.859055289186339, 'max_bin': 384, 'n_estimators': 2820}, SMAPE: 11.14
건물 26 → {'max_leaves': 453, 'learning_rate': 0.0015444560682636894, 'subsample': 0.8287177280012834, 'colsample_bytree': 0.8926691028977085, 'lambda': 0.9553004227789768, 'alpha': 0.7520900849796, 'max_bin': 1406, 'n_estimators': 4995}, SMAPE: 8.32
건물 27 → {'max_leaves': 50, 'learning_rate': 0.0027489119625797903, 'subsample': 0.5443964275131294, 'colsample_bytree': 0.8865435299904045, 'lambda': 0.012240644044031194, 'alpha': 7.186411020761515, 'max_bin': 594, 'n_estimators': 3227}, SMAPE: 10.65
건물 28 → {'max_leaves': 181, 'learning_rate': 0.001520149289500025, 'subsample': 0.5237398104014767, 'colsample_bytree': 0.6163471735002414, 'lambda': 0.014322742941319305, 'alpha': 0.4

Exception ignored in: <function ResourceTracker.__del__ at 0x107101bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1026d9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x104be9bc0>
Traceback (most recent call last

In [29]:
import json

SAVE_PATH = "xgb_val_100.json"

with open(SAVE_PATH, "r") as f:
    best_params_all = json.load(f)

print(best_params_all.keys())  # 건물 번호들 확인

dict_keys(['6', '8', '1', '5', '3', '7', '2', '4', '10', '9', '12', '15', '14', '11', '16', '17', '13', '18', '23', '20', '22', '24', '19', '21', '25', '28', '27', '32', '30', '31', '29', '26', '33', '35', '34', '36', '39', '41', '38', '37', '40', '47', '44', '42', '46', '43', '45', '48', '50', '49', '51', '56', '52', '53', '58', '55', '57', '61', '54', '59', '60', '66', '62', '64', '63', '67', '65', '68', '69', '71', '70', '73', '77', '72', '76', '74', '75', '78', '79', '83', '80', '81', '84', '82', '86', '85', '90', '88', '91', '89', '87', '93', '94', '92', '96', '95', '97', '98', '99', '100'])


In [30]:
SAVE_MODEL_DIR = "XGBbuilding_val"
os.makedirs(SAVE_MODEL_DIR, exist_ok=True)

In [45]:
train_temp1 = pd.concat([train_all,train_df['datetime']],axis=1)
valid_temp1 = pd.concat([valid_all,valid_df['datetime']],axis=1)

In [47]:
valid_temp1

,건물번호,기온(°C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),시간,요일,weekend,hour_sin,hour_cos,dow_sin,dow_cos,hour_sin_dow_sin,hour_sin_dow_cos,hour_cos_dow_sin,hour_cos_dow_cos,불쾌지수,august,datetime
0,1,27.3,0.0,1.5,83.0,4036.38,0,6,1,0.000000,1.000000,-0.781831,0.623490,-0.000000,0.000000,-0.781831,0.623490,26.10320,1,2024-08-18 00:00:00
1,1,27.3,0.0,1.2,84.0,3923.43,1,6,1,0.258819,0.965926,-0.781831,0.623490,-0.202353,0.161371,-0.755191,0.602245,26.17360,1,2024-08-18 01:00:00
2,1,27.2,0.0,2.3,85.0,3917.10,2,6,1,0.500000,0.866025,-0.781831,0.623490,-0.390916,0.311745,-0.677086,0.539958,26.15225,1,2024-08-18 02:00:00
3,1,27.1,0.0,2.1,84.0,3875.88,3,6,1,0.707107,0.707107,-0.781831,0.623490,-0.552838,0.440874,-0.552838,0.440874,25.99120,1,2024-08-18 03:00:00
4,1,27.1,0.0,1.6,83.0,4023.48,4,6,1,0.866025,0.500000,-0.781831,0.623490,-0.677086,0.539958,-0.390916,0.311745,25.92190,1,2024-08-18 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,29.1,0.0,4.4,76.0,3276.00,19,5,1,-0.965926,0.258819,-0.974928,-0.222521,0.941708,0.214939,-0.252330,-0.057593,27.17280,1,2024-08-24 19:00:00
16796,100,28.6,0.0,3.7,74.0,3197.52,20,5,1,-0.866025,0.500000,-0.974928,-0.222521,0.844312,0.192709,-0.487464,-0.111260,26.58370,1,2024-08-24 20:00:00
16797,100,28.3,0.0,2.9,74.0,3006.60,21,5,1,-0.707107,0.707107,-0.974928,-0.222521,0.689378,0.157346,-0.689378,-0.157346,26.32660,1,2024-08-24 21:00:00
16798,100,28.0,0.0,1.7,76.0,2649.72,22,5,1,-0.500000,0.866025,-0.974928,-0.222521,0.487464,0.111260,-0.844312,-0.192709,26.21800,1,2024-08-24 22:00:00


In [52]:
train_temp_full = pd.concat([train_temp1, valid_temp1], ignore_index=True)

#건물번호, datetime 기준으로 정렬
train_final = train_temp_full.sort_values(by=['건물번호', 'datetime']).reset_index(drop=True).drop(['datetime'],axis=1)

In [59]:
train_final

,건물번호,기온(°C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),시간,요일,weekend,hour_sin,hour_cos,dow_sin,dow_cos,hour_sin_dow_sin,hour_sin_dow_cos,hour_cos_dow_sin,hour_cos_dow_cos,불쾌지수,august
0,1,18.3,0.0,2.6,82.0,5794.80,0,5,1,0.000000,1.000000,-0.974928,-0.222521,-0.000000,-0.000000,-0.974928,-0.222521,17.92380,0
1,1,18.3,0.0,2.7,82.0,5591.85,1,5,1,0.258819,0.965926,-0.974928,-0.222521,-0.252330,-0.057593,-0.941708,-0.214939,17.92380,0
2,1,18.1,0.0,2.6,80.0,5338.17,2,5,1,0.500000,0.866025,-0.974928,-0.222521,-0.487464,-0.111260,-0.844312,-0.192709,17.70400,0
3,1,18.0,0.0,2.6,81.0,4554.42,3,5,1,0.707107,0.707107,-0.974928,-0.222521,-0.689378,-0.157346,-0.689378,-0.157346,17.63425,0
4,1,17.8,0.0,1.3,81.0,3602.25,4,5,1,0.866025,0.500000,-0.974928,-0.222521,-0.844312,-0.192709,-0.487464,-0.111260,17.45515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,29.1,0.0,4.4,76.0,3276.00,19,5,1,-0.965926,0.258819,-0.974928,-0.222521,0.941708,0.214939,-0.252330,-0.057593,27.17280,1
203996,100,28.6,0.0,3.7,74.0,3197.52,20,5,1,-0.866025,0.500000,-0.974928,-0.222521,0.844312,0.192709,-0.487464,-0.111260,26.58370,1
203997,100,28.3,0.0,2.9,74.0,3006.60,21,5,1,-0.707107,0.707107,-0.974928,-0.222521,0.689378,0.157346,-0.689378,-0.157346,26.32660,1
203998,100,28.0,0.0,1.7,76.0,2649.72,22,5,1,-0.500000,0.866025,-0.974928,-0.222521,0.487464,0.111260,-0.844312,-0.192709,26.21800,1


In [54]:
import joblib
import os

for b_id, info in best_params_all.items():
    print(f"\n=== [건물 {b_id}] 학습 시작 ===")

    # 파라미터 불러오기
    best_params = info["best_params"]

    # 데이터 분리
    bld_train = train_final[train_final["건물번호"] == int(b_id)]
    X_train = bld_train.drop(columns=["건물번호", "전력소비량(kWh)"])
    y_train = bld_train["전력소비량(kWh)"]

    # 모델 학습
    model = xgb.XGBRegressor(**best_params,enable_categorical= True,tree_method="hist")
    model.fit(X_train, y_train)

    # 저장
    model_path = os.path.join(SAVE_MODEL_DIR, f"building_{b_id}.pkl")
    joblib.dump(model, model_path)
    print(f"[저장 완료] {model_path}")


=== [건물 6] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_6.pkl

=== [건물 8] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_8.pkl

=== [건물 1] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_1.pkl

=== [건물 5] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_5.pkl

=== [건물 3] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_3.pkl

=== [건물 7] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_7.pkl

=== [건물 2] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_2.pkl

=== [건물 4] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_4.pkl

=== [건물 10] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_10.pkl

=== [건물 9] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_9.pkl

=== [건물 12] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_12.pkl

=== [건물 15] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_15.pkl

=== [건물 14] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_14.pkl

=== [건물 11] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_11.pkl

=== [건물 16] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_16.pkl

=== [건물 17] 학습 시작 ===
[저장 완료] XGBbuilding_val/building_17.pkl

=== [건물 1

In [55]:
FINAL_MODEL_PATH = "all_buildings_model_val.pkl"

# === 1. 모든 개별 모델 불러오기 ===
model_dict = {}
for fname in os.listdir(SAVE_MODEL_DIR):
    if fname.endswith(".pkl"):
        b_id = int(fname.split("_")[1].split(".")[0])  # building_7.pkl → 7
        model_path = os.path.join(SAVE_MODEL_DIR, fname)
        model_dict[b_id] = joblib.load(model_path)

print(f"총 {len(model_dict)}개 건물 모델 로드 완료!")

# === 2. dict 자체를 저장 ===
joblib.dump(model_dict, FINAL_MODEL_PATH)
print(f"모든 건물 모델을 하나의 파일로 저장 완료 → {FINAL_MODEL_PATH}")

총 100개 건물 모델 로드 완료!
모든 건물 모델을 하나의 파일로 저장 완료 → all_buildings_model_val.pkl


In [56]:
class BuildingPredictor:
    def __init__(self, model_path="all_buildings_model_val.pkl"):
        # 모델 딕셔너리 불러오기
        self.model_dict = joblib.load(model_path)
    
    def predict(self, df: pd.DataFrame) -> pd.Series:
        """
        df: 건물번호 + 피처들이 들어있는 DataFrame
            (반드시 '건물번호'와 '전력소비량(kWh)' 컬럼이 존재해야 함)
        
        return: 예측값이 들어있는 Series (index는 df와 동일)
        """
        preds = pd.Series(index=df.index, dtype=float)

        for b_id, group in df.groupby("건물번호"):
            if b_id not in self.model_dict:
                raise ValueError(f"건물 {b_id} 모델이 없습니다!")

            model = self.model_dict[b_id]

            # feature만 추출
            X = group.drop(columns=["건물번호"])
            preds.loc[group.index] = model.predict(X)
        
        return preds

In [58]:
predictor = BuildingPredictor("all_buildings_model_val.pkl")

In [63]:
y_pred_train = predictor.predict(train_final.drop(['전력소비량(kWh)'],axis=1))
y_pred_test = predictor.predict(test_all)

In [71]:
train_check = pd.concat([train_final,train_temp_full[['datetime']]],axis=1)
test_check = pd.concat([test_all,test_df[['datetime']]],axis=1)

In [74]:
train_check['y_pred'] = predictor.predict(train_final.drop(['전력소비량(kWh)'],axis=1))